In [17]:
import pandas as pd
import timeit
from numpy import *
start = timeit.default_timer()
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None
                , na_values = "?")
df.columns = ['age', 'workclass','fnlwgt','education no', 'education', 'marital status', 'occupation', 'relationship', 'race',
              'sex', 'capital gain', 'capital loss','hours per week','native country', 'class']
print df.head()
df.drop(['education no', 'fnlwgt'], axis = 1)
def transformAge(d1):
    if d1 <= 25: return 'Young'
    elif 26 < d1 <= 45 : return 'Middle Aged'
    elif 46<= d1 <= 65 : return 'Senior'
    else : return 'Old'
df["Age"]= df['age'].map(transformAge)    

def transformHours(d1):
    if d1 <= 25: return 'Part time'
    elif 25 < d1 <= 40 : return 'Full time'
    elif 40 < d1 <= 60 : return 'Overtime'
    else: return 'Too much'
df["Hours per week"]= df['hours per week'].map(transformHours)
df.drop(['hours per week'], axis = 1)
import numpy as np
capital = []
for index, row in df.iterrows():
    if (row['capital loss'] != 0):
        capital.append(row['capital loss'])
array1 = np.array(capital)
print np.median(array1)
capital2 = []
for index, row in df.iterrows():
    if (row['capital gain'] != 0):
        capital2.append(row['capital gain'])
array2 = np.array(capital2)
print np.median(array2)
def transformCapitalloss(d1):
    if (d1 == 0): return 'nil'
    elif (d1 <= 1887): return 'Low'
    else : return 'High'
df['Capital loss'] = df['capital loss'].map(transformCapitalloss)

def transformCapitalgain(d1):
    if (d1 == 0): return 'nil'
    elif (d1 <= 7298): return 'Low'
    else : return 'High'
df['Capital gain'] = df['capital gain'].map(transformCapitalloss)

df_cleaned = df.drop(['age','fnlwgt','education','hours per week','capital loss', 'capital gain'], axis = 1)
print df_cleaned.head()      
def loadDataSet():
    return df_cleaned.values.tolist()
transactions = df_cleaned.values.tolist()

   age          workclass  fnlwgt education no  education  \
0   39          State-gov   77516    Bachelors         13   
1   50   Self-emp-not-inc   83311    Bachelors         13   
2   38            Private  215646      HS-grad          9   
3   53            Private  234721         11th          7   
4   28            Private  338409    Bachelors         13   

        marital status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital gain  capital loss  hours per week  native country   class  
0          2174             0              40   United-States   <=50K  
1             

In [15]:
import itertools


class TreeNode(object):
    

    def __init__(self, value, count, parent):
        
        self.value = value
        self.count = count
        self.parent = parent
        self.link = None
        self.children = []

    def has_child(self, value):
        
        for node in self.children:
            if node.value == value:
                return True

        return False

    def get_child(self, value):
       
        for node in self.children:
            if node.value == value:
                return node

        return None

    def add_child(self, value):
        
        child = TreeNode(value, 1, self)
        self.children.append(child)
        return child


class TreeGrowth(object):
    
#building the fp tree
    def __init__(self, transactions, threshold, root_value, root_count):
        
        self.frequent = self.find_frequent_items(transactions, threshold)
        self.headers = self.build_headertable(self.frequent)
        self.root = self.build_fptree(
            transactions, root_value,
            root_count, self.frequent, self.headers)

    
    def find_frequent_items(self, transactions, threshold):
        
        items = {}

        for transaction in transactions:
            for item in transaction:
                if item in items:
                    items[item] += 1
                else:
                    items[item] = 1

        for key in list(items.keys()):
            if items[key] < threshold:
                del items[key]

        return items

   
    def build_headertable(self, frequent):
        
        headers = {}
        for key in frequent.keys():
            headers[key] = None

        return headers

    def build_fptree(self, transactions, root_value,
                     root_count, frequent, headers):
       
        root = TreeNode(root_value, root_count, None)

        for transaction in transactions:
            sorted_items = [x for x in transaction if x in frequent]
            sorted_items.sort(key=lambda x: frequent[x], reverse=True)
            if len(sorted_items) > 0:
                self.insertnodes(sorted_items, root, headers)

        return root

    def insertnodes(self, items, node, headers):
        
        first = items[0]
        child = node.get_child(first)
        if child is not None:
            child.count += 1
        else:
            # Add new child.
            child = node.add_child(first)

            # Link it to header structure.
            if headers[first] is None:
                headers[first] = child
            else:
                current = headers[first]
                while current.link is not None:
                    current = current.link
                current.link = child

        # Call function recursively.
        remaining_items = items[1:]
        if len(remaining_items) > 0:
            self.insert_tree(remaining_items, child, headers)
#Mining the tree
    def tree_has_single_path(self, node):
        
        num_children = len(node.children)
        if num_children > 1:
            return False
        elif num_children == 0:
            return True
        else:
            return True and self.tree_has_single_path(node.children[0])

    def mine_patterns(self, threshold):
       
        if self.tree_has_single_path(self.root):
            return self.generatepatterns()
        else:
            return self.zip_patterns(self.minesubtrees(threshold))

    def zip_patterns(self, patterns):
        
        suffix = self.root.value

        if suffix is not None:
            # We are in a conditional tree.
            new_patterns = {}
            for key in patterns.keys():
                new_patterns[tuple(sorted(list(key) + [suffix]))] = patterns[key]

            return new_patterns

        return patterns

    def generatepatterns(self):
        
        patterns = {}
        items = self.frequent.keys()
        
        if self.root.value is None:
            suffix_value = []
        else:
            suffix_value = [self.root.value]
            patterns[tuple(suffix_value)] = self.root.count

        for i in range(1, len(items) + 1):
            for subset in itertools.combinations(items, i):
                pattern = tuple(sorted(list(subset) + suffix_value))
                patterns[pattern] = \
                    min([self.frequent[x] for x in subset])

        return patterns

    def minesubtrees(self, threshold):
        
        patterns = {}
        mining_order = sorted(self.frequent.keys(),
                              key=lambda x: self.frequent[x])

        # Get items in tree in reverse order of occurrences.
        for item in mining_order:
            suffixes = []
            conditional_tree_input = []
            node = self.headers[item]

            # Follow node links to get a list of
            # all occurrences of a certain item.
            while node is not None:
                suffixes.append(node)
                node = node.link

            # For each occurrence of the item, 
            # trace the path back to the root node.
            for suffix in suffixes:
                frequency = suffix.count
                path = []
                parent = suffix.parent

                while parent.parent is not None:
                    path.append(parent.value)
                    parent = parent.parent

                for i in range(frequency):
                    conditional_tree_input.append(path)

            
            subtree = TreeGrowth(conditional_tree_input, threshold,
                             item, self.frequent[item])
            subtree_patterns = subtree.mine_patterns(threshold)

            for pattern in subtree_patterns.keys():
                if pattern in patterns:
                    patterns[pattern] += subtree_patterns[pattern]
                else:
                    patterns[pattern] = subtree_patterns[pattern]

        return patterns


def find_frequent_patterns(transactions, support_threshold):
    
    tree = TreeGrowth(transactions, support_threshold, None, None)
    return tree.mine_patterns(support_threshold)
patterns = find_frequent_patterns(transactions, 10000)
patterns

{(' <=50K', ' Male', ' Private'): 10707,
 (' <=50K', ' Male', ' Private', 'nil'): 20623,
 (' <=50K', ' Male', ' United-States'): 13389,
 (' <=50K', ' Male', ' United-States', ' White'): 11913,
 (' <=50K', ' Male', ' United-States', ' White', 'nil'): 22865,
 (' <=50K', ' Male', ' United-States', 'nil'): 25730,
 (' <=50K', ' Male', ' White', 'nil'): 25135,
 (' <=50K', ' Male', ' White', 'nil', 'nil'): 12050,
 (' <=50K', ' Male', 'nil'): 29091,
 (' <=50K', ' Male', 'nil', 'nil'): 13963,
 (' <=50K', ' Never-married'): 10192,
 (' <=50K', ' Never-married', 'nil'): 19809,
 (' <=50K', ' Private', ' United-States'): 15594,
 (' <=50K', ' Private', ' United-States', ' White'): 13452,
 (' <=50K', ' Private', ' United-States', ' White', 'nil'): 25936,
 (' <=50K', ' Private', ' United-States', 'nil'): 30108,
 (' <=50K', ' Private', ' White', 'nil'): 28693,
 (' <=50K', ' Private', ' White', 'nil', 'nil'): 13821,
 (' <=50K', ' Private', 'Full time'): 11383,
 (' <=50K', ' Private', 'Full time', 'nil'):

In [20]:
stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time) + "seconds")

Program Executed in 7.97568352189seconds
